<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/HW9_Boosting_XGBoost%2C_LightGBM%2C_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework:

- use the dataset and validation approach which you used for previous homeworks
- train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
- compare results between them and previously trained random forest

# Init

In [ ]:
!pip install --upgrade pandas==1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, accuracy_score
from sklearn import metrics
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

train_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/train.csv")
test_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/test.csv")

In [ ]:
random_state = 42

In [ ]:
def generate(df,age_median,fare_median):
  useless_features = ['Name','Ticket','Cabin']
  data_cleaned = df
  data_cleaned = data_cleaned.drop(columns = useless_features)

  # generate binary values using get_dummies
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Sex'],prefix=["Sex"])
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Embarked'],prefix=["Embarked"])

  # Check for NaN values in the DataFrame
  nan_mask = data_cleaned.isnull()
  # Count the number of NaN values in each column
  nan_count_per_column = data_cleaned.isnull().sum()

  data_cleaned['Age'] = data_cleaned['Age'].fillna(age_median)
  data_cleaned['Fare'] = data_cleaned['Fare'].fillna(fare_median)

  return data_cleaned


In [ ]:
not_features = ['PassengerId', 'Survived']

In [ ]:
# get medians that will fill NaNs in generate func
age_median = train_full['Age'].median()
print(age_median)
fare_median = train_full['Fare'].median()
print(fare_median)

NameError: ignored

In [ ]:
train = generate(train_full,age_median=age_median,fare_median=fare_median)
print(train)

X_train = train.drop(not_features, axis = 1)
y_train = train['Survived']

In [ ]:
# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [ ]:
# Define the custom scoring function with the desired beta value
beta = 0.5
custom_scorer = make_scorer(fbeta_score, beta=beta)

#XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param_grid = {
    #default
    "objective": "binary:logistic",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "eval_metric": "auc",

    # regularization parameters
    "max_leaves": 32,
    "subsample": 0.7,
    "colsample_bytree": 0.7,

    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param_grid, xgb_train, num_rounds, early_stopping_rounds=50,folds=skf, verbose_eval=20)

In [ ]:
param_grid = {
    #default
    "objective": ["binary:logistic"],
    "eta": [0.01, 0.05],
    "verbosity": [0],
    "nthread": [10],
    "random_seed": [random_state],


    # regularization parameters
    "max_leaves": [16, 32, 64],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],

    #lightgbm approach
    "tree_method": ["hist"],
    "grow_policy": ["lossguide"]
}

In [ ]:
# xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',silent=True, nthread=1)

xgb_model = xgb.XGBClassifier()
grid_search = GridSearchCV(xgb_model, param_grid, scoring=custom_scorer, cv=skf )

# Perform grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

# LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:


parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 50
}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, early_stopping_rounds=50, verbose_eval=100, eval_train_metric=True)